<a href="https://colab.research.google.com/github/jamandgar/NFL_Data_Analysis/blob/master/NFL_HallOfFame_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#CSV Data Collected from Pro Football Reference: 
https://www.pro-football-reference.com/play-index/psl_finder.cgi

###League-Wide Data


In [38]:
hof_players = pd.read_csv("All_HOF.csv")
mvp_players_nfl = pd.read_csv("All_MVP-NFL.csv")
mvp_players_sb = pd.read_csv("All_MVP-SB.csv")

###Individual Player Data
Data was collected for quarterbacks (QB) who played a minimum of 10 games and retired from the National Football League (NFL) between 1920, its inaugural year, and 2014, the most recent year - as of 2020 - a player could have played and still be eligible for Hall of Fame (HOF) induction.

#####Quarterbacks

In [39]:
#Super Bowl (SB) appearances and victories (typically tracked for just the
#quarterback position)
qb_sb_games = pd.read_csv("QB_Championships.csv")

#Passing data
qb_passing = pd.read_csv("QB_Passing.csv")

#Combined passing, rushing, and/or receiving data
qb_totals = pd.read_csv("QB_Totals.csv")

#Other data related to in-game statistics, e.g. Pro Bowl appearances
qb_misc = pd.read_csv("QB_Miscellaneous.csv")

#Data Cleaning

In [40]:
hof_qbs = hof_players[hof_players['Pos'] == 'QB']
